# Kryptos


## Practical introduction to brute-force decryption of K1-K3
*with very simple Python code and some statistics about English language* 



### Prerequisites

Download files (count_2l.txt and count_1w.txt) from Peter Norvig's website: https://norvig.com/ngrams/

These files are derived from the Google Web Trillion Word Corpus, (c) Google and free to use for educational purpose

NB: if you do not have wget on your computer, you can skip the following cell and download the files manually into the current directory




In [1]:
!wget https://norvig.com/ngrams/count_2l.txt
!wget https://norvig.com/ngrams/count_1w.txt


--2022-01-24 21:53:03--  https://norvig.com/ngrams/count_2l.txt
Resolving norvig.com (norvig.com)... 158.106.138.13
Connecting to norvig.com (norvig.com)|158.106.138.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9270 (9.1K) [text/plain]
Saving to: ‘count_2l.txt’

count_2l.txt        100%[===================>]   9.05K  --.-KB/s    in 0s      

2022-01-24 21:53:04 (47.3 MB/s) - ‘count_2l.txt’ saved [9270/9270]

--2022-01-24 21:53:04--  https://norvig.com/ngrams/count_1w.txt
Resolving norvig.com (norvig.com)... 158.106.138.13
Connecting to norvig.com (norvig.com)|158.106.138.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4956241 (4.7M) [text/plain]
Saving to: ‘count_1w.txt’

count_1w.txt        100%[===================>]   4.73M  5.13MB/s    in 0.9s    

2022-01-24 21:53:05 (5.13 MB/s) - ‘count_1w.txt’ saved [4956241/4956241]



### Evaluation function

Load count_2l.txt file into a dictionnary where a bigram is key to a value that represents its number of occurences:

In [2]:
NORM_FACTOR = 2800000 # normalization that ponders weights relatively to the rarest bigram
with open('./count_2l.txt') as f:
    weights = {line.split()[0].upper(): int(line.split()[1]) // NORM_FACTOR for line in f}


Rarest and most frequent bigrams:

In [3]:
print(weights['JQ'])
print(weights['IN'])

1
48147


Build a very simple evaluation function that tries to detect if a string actually contains a correct english sentence. In a single pass, a score is calculated as the addition of all bigram weights:

In [4]:
def score(sentence):
    duos = [sentence[i:i+2] for i in range(len(sentence) - 1)]
    total = 0
    for duo in duos:
            if duo in weights: total += weights[duo]
    return total // len(sentence)


### K1

The ciphertext is:

In [5]:
K1 = '''
EMUFPHZLRFAXYUSDJKZLDKRNSHGNFIVJ
YQTQUXQBQVYUVLLTREVJYQTMKYRDMFD
'''.replace('\n','').replace('?','')

In [6]:
len(K1)

63

The sculpture hints at a Vigenère table where the alphabet letters are transposed with the KRYPTOS keyword:

In [7]:
ALPHA_K = "KRYPTOSABCDEFGHIJLMNQUVWXZ"

Define a utility function that builds the equivalent index, a 26-integer list indicating that for exemple the character 'A' represented by slot 0 can be found at slot 7 of the keyed alphabet:

In [8]:
def alphabet_index(alphabet):
    idx = [-1] * 26
    for i, letter in enumerate(alphabet):
        idx[ord(letter) - ord('A')] = i
    return idx


Both will be passed to the Vigenère function:

In [9]:
K = (ALPHA_K, alphabet_index(ALPHA_K))


Define the Vigenère function. The key is used to shift the keyed alphabet:

In [10]:
def vigenere(alphabet, key, ciphertext):
    plaintext = ''
    letters, idx = alphabet
    for i, letter in enumerate(ciphertext):
        p = key[i % len(key)]
        offset = idx[ord(p) - ord('A')]
        l = letters[(idx[ord(letter) - ord('A')] + 26 - offset) % 26]
        plaintext += l
    return plaintext


Load count_1w.txt file into a list of english words, truncated to 150000 most frequent words:

In [11]:
with open('./count_1w.txt') as f:
    words = [line.split()[0].upper() for line in f][:150000]


Make the assumption that the keyword is at least 8-character long. It will cut the possibilities in half:

In [12]:
long_words = [w for w in words if len(w) > 7]
len(long_words)

63883

Apply the Vigenère function with all the possible keywords and calculate the score of each output string:

In [13]:
s = [score(vigenere(K, candidate, K1)) for candidate in long_words]

Look for the max score and then for the corresponding word:

In [14]:
keyword = long_words[s.index(max(s))]
keyword

'PALIMPSEST'

Word PALIMPSEST looks like the best candidate. Verify if this keyword actually produce a correct sentence:

In [15]:
vigenere(K, keyword, K1)

'BETWEENSUBTLESHADINGANDTHEABSENCEOFLIGHTLIESTHENUANCEOFIQLUSION'


> *Between subtle shading and the absence of light lies the nuance of illusion*

### K2

K1 has shown that 60 characters is enough to calculate a useful score. K2 is very similar to K1: apply the same logic while truncating K2 in orderto speed up the process.

In [16]:
K2 = '''
VFPJUDEEHZWETZYVGWHKKQETGFQJNCE
GGWHKK?DQMCPFQZDQMMIAGPFXHQRLG
TIMVMZJANQLVKQEDAGDVFRPJUNGEUNA
QZGZLECGYUXUEENJTBJLBQCRTBJDFHRR
YIZETKZEMVDUFKSJHKFWHKUWQLSZFTI
HHDDDUVH?DWKBFUFPWNTDFIYCUQZERE
EVLDKFEZMOQQJLTTUGSYQPFEUNLAVIDX
FLGGTEZ?FKZBSFDQVGOGIPUFXHHDRKF
FHQNTGPUAECNUVPDJMQCLQUMUNEDFQ
ELZZVRRGKFFVOEEXBDMVPNFQXEZLGRE
DNQFMPNZGLFLPMRJQYALMGNUVPDXVKP
DQUMEBEDMHDAFMJGZNUPLGESWJLLAETG
'''.replace('\n','').replace('?','')

In [17]:
s = [score(vigenere(K, candidate, K2[:60])) for candidate in long_words]
keyword = long_words[s.index(max(s))]
keyword

'ABSCISSA'

In [18]:
vigenere(K, keyword, K2)

'ITWASTOTALLYINVISIBLEHOWSTHATPOSSIBLETHEYUSEDTHEEARTHSMAGNETICFIELDXTHEINFORMATIONWASGATHEREDANDTRANSMITTEDUNDERGRUUNDTOANUNKNOWNLOCATIONXDOESLANGLEYKNOWABOUTTHISTHEYSHOULDITSBURIEDOUTTHERESOMEWHEREXWHOKNOWSTHEEXACTLOCATIONONLYWWTHISWASHISLASTMESSAGEXTHIRTYEIGHTDEGREESFIFTYSEVENMINUTESSIXPOINTFIVESECONDSNORTHSEVENTYSEVENDEGREESEIGHTMINUTESFORTYFOURSECONDSWESTXLAYERTWO'

> *It was totally invisible (...)*

### K3

K3 is a transposition. Letters are not not obfuscated using a keyed alphabet: they are shifted/reordered with a pattern.

In [19]:
K3 = '''
ENDYAHROHNLSRHEOCPTEOIBIDYSHNAIA
CHTNREYULDSLLSLLNOHSNOSMRWXMNE
TPRNGATIHNRARPESLNNELEBLPIIACAE
WMTWNDITEENRAHCTENEUDRETNHAEOE
TFOLSEDTIWENHAEIOYTEYQHEENCTAYCR
EIFTBRSPAMHHEWENATAMATEGYEERLB
TEEFOASFIOTUETUAEOTOARMAEERTNRTI
BSEDDNIAAHTTMSTEWPIEROAGRIEWFEB
AECTDDHILCEIHSITEGOEAOSDDRYDLORIT
RKLMLEHAGTDHARDPNEOHMGFMFEUHE
ECDMRIPFEIMEHNLSSTTRTVDOHW?
'''.replace('\n','')


Define a transposition function that simply shifts letters using a serie of indexes built with modular artithmetic.
For example, if the length of the text is 11 and the chosen key step is 7, for the first 3 letters the indexes are :
- 6 (the 7th letter)
- 2 (6 + 7 mod 11)
- 9 (6 + 2*7 mod 11)
```
"ABCDEFGHIJK" => "GCJFBIEAHDK"
   2   6  9       629
```
etc...

In [20]:
def transposition(step, ciphertext):
    plaintext = ''
    size = len(ciphertext)
    for i in range(size):
        plaintext += ciphertext[(step - 1 + i * step) % size]
    return plaintext


In [21]:
s = [score(transposition(i, K3)) for i in range(len(K3))]

In [22]:
shift_number = s.index(max(s))
shift_number

192

In [23]:
transposition(shift_number, K3)

'SLOWLYDESPARATLYSLOWLYTHEREMAINSOFPASSAGEDEBRISTHATENCUMBEREDTHELOWERPARTOFTHEDOORWAYWASREMOVEDWITHTREMBLINGHANDSIMADEATINYBREACHINTHEUPPERLEFTHANDCORNERANDTHENWIDENINGTHEHOLEALITTLEIINSERTEDTHECANDLEANDPEEREDINTHEHOTAIRESCAPINGFROMTHECHAMBERCAUSEDTHEFLAMETOFLICKERBUTPRESENTLYDETAILSOFTHEROOMWITHINEMERGEDFROMTHEMISTXCANYOUSEEANYTHINGQ?'

> *Slowly desperatly slowly (...)*

These words may relate to the progress made on K4 for the last 20 years ;-) 